In [1]:
# TODO
# Make function to plot aggregated evolving contours and store files
# Modify find contours function or use if statement to achieve less than 10% of contours intersecting with buffer
# Import/read ATL11 (try h5coro)
# Add ATL11 overlay
# Add something along the lines of code below to illustrate lake area/outline:
# # Plot polygons in the GeoDataFrame
# gdf.plot(ax=ax, color='lightblue', edgecolor='black', linewidth=1, label='Polygon Area')

# # Optionally, if you want to plot just the boundaries with a different style
# gdf.boundary.plot(ax=ax, color='red', linewidth=2, label='Polygon Boundary')

In [2]:
# Code to Fig. S1 of Sauthoff and others, 2024
# This code run continental-scale operations on multiple datasets and
# requires a 64 GB server or local memory
#
# Written 2023-07-11 by W. Sauthoff (sauthoff@mines.edu)

# Set up computing environment

In [3]:
%pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [122]:
# Import libraries
import datetime
import earthaccess
import geopandas as gpd
from IPython.display import clear_output
import matplotlib
import matplotlib.cm as cm
from matplotlib.collections import LineCollection
import matplotlib.colors as colors
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import os
from os import path
from pyproj import CRS, Transformer
import rioxarray
from rioxarray.exceptions import NoDataInBounds
from shapely.geometry import box, Polygon
from shapely.ops import unary_union
from skimage import measure
import xarray as xr

# Magic functions
%matplotlib widget

from IPython.display import Audio, display
def play_sound():
    display(Audio(url="https://example.com/your-sound-file.mp3", autoplay=True))

# Define data directories dependent on home environment
# Replace with your directory file paths
if os.getenv('HOME') == '/home/jovyan':
    DATA_DIR = '/home/jovyan/data'
    SCRIPT_DIR = '/home/jovyan/repos_my/script_dir'
    OUTPUT_DIR = '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb'

# Define constants and coordinate transforms for the geodesic area calculation
CRS_LL = "EPSG:4326" # wgs84 in lon,lat
GEOD = CRS(CRS_LL).get_geod() # geod object for calculating geodesic area on defined ellipsoid
CRS_XY = "EPSG:3031" # Antarctic Polar Stereographic in x, y
XY_TO_LL = Transformer.from_crs(CRS_XY, CRS_LL, always_xy = True) # make coord transformer

# Change default font to increase font size
plt.rcParams.update({'font.size': 8})

# Functions

In [5]:
def read_field(group, field):
    """
    generic field-reading function
    """
    data=np.array(group[field])
    bad=(data==group[field].attrs['_FillValue'])
    data[bad]=np.NaN
    return data

In [6]:
def read_ATL11(filename, pair):
    """
    ATL11 reader
    """
    with h5py.File(filename,'r') as h5f:
        longitude=read_field(h5f[pair],'longitude')
        latitude=read_field(h5f[pair],'latitude')
        h_corr=read_field(h5f[pair],'h_corr')
        h_corr_sigma=read_field(h5f[pair],'h_corr_sigma')
        h_corr_sigma_s=read_field(h5f[pair],'h_corr_sigma_systematic')
        quality=np.array(h5f[pair]['quality_summary'])
    for col in range(h_corr.shape[1]):
        h_corr[quality==1]=np.NaN
    # return the values
    return longitude, latitude, h_corr, np.sqrt(h_corr_sigma**2+h_corr_sigma_s**2)

In [7]:
def datetime64_to_fractional_year(date):
    """
    Convert a numpy.datetime64 object into a fractional year, rounded to 0, .25, .5, or .75.
    """
    year = date.astype('datetime64[Y]').astype(int) + 1970
    start_of_year = np.datetime64(f'{year}-01-01')
    start_of_next_year = np.datetime64(f'{year + 1}-01-01')
    year_length = (start_of_next_year - start_of_year).astype('timedelta64[D]').astype(int)
    day_of_year = (date - start_of_year).astype('timedelta64[D]').astype(int)
    fractional_year = year + day_of_year / year_length

    # Round to nearest quarter
    rounded_fractional_year = round(fractional_year * 4) / 4
    return rounded_fractional_year

# # Example usage
# date = np.datetime64('2024-01-05')
# fractional_year = datetime64_to_fractional_year(date)

In [8]:
# def date_to_fractional_year(date):
#     """
#     Convert a datetime.date or datetime.datetime object into a fractional year
#     """
#     year = date.year
#     start_of_year = datetime.date(year, 1, 1)
#     start_of_next_year = datetime.date(year + 1, 1, 1)
#     year_length = (start_of_next_year - start_of_year).days
#     day_of_year = (date - start_of_year).days
#     fractional_year = year + day_of_year / year_length
#     return fractional_year

# # # Example usage with a datetime.date object
# # date = datetime.date(2024, 1, 5)
# # fractional_year = date_to_fractional_year(date)
# # fractional_year

In [9]:
def find_intersections(gdf1, gdf2):
    # Create an empty list to store the results
    intersections = []

    # Iterate over each geometry in gdf1
    for index1, geom1 in gdf1.geometry.items():
        # Compare with each geometry in gdf2
        for index2, geom2 in gdf2.geometry.items():
            if geom1.intersects(geom2):
                # If they intersect, add the indices to the list
                intersections.append((index1, index2))

    return intersections

In [10]:
# def unify_polygons(polygons):
#     """
#     Unifies a list of polygons into a single polygon.

#     :param polygons: A list of polygon geometries.
#     :return: A unified polygon geometry.
#     """
#     if not polygons:
#         return None  # or raise an error if appropriate

#     # Extract the geometry values from the GeoSeries and put them in a list
#     geometry_list = [polygon.geometry.values for polygon in polygons]

#     # Use unary_union to handle the union of multiple geometries
#     # union_polygon = unary_union(geometry_list)
#     union_polygon = unary_union(geometry_list)

#     return union_polygon

# # # Create a list of polygons
# # polygons = [polygon1, polygon2, polygon3]

# # # Get the unified polygon
# # unified_polygon = unify_polygons(polygons)

In [11]:
# def unify_polygons(polygons):
#     """
#     Unifies a list of polygons into a single polygon.

#     :param polygons: A list of polygon geometries.
#     :return: A unified polygon geometry.
#     """
#     if not polygons:
#         return None  # or raise an error if appropriate

#     # Extract the geometry values from the GeoSeries and put them in a list
#     geometry_list = [polygon.geometry.values for polygon in polygons]

#     # Use unary_union to handle the union of multiple geometries
#     # union_polygon = unary_union(geometry_list)
#     union_polygon = unary_union(geometry_list)
    
#     # Convert to geodataframe
#     union_polygon_gdf = gpd.GeoDataFrame({'geometry': [union_polygon]})
    
#     # Set CRS
#     union_polygon_gdf.geometry.crs = 'EPSG:3031'

#     return union_polygon_gdf

# # # Create a list of polygons
# # polygons = [polygon1, polygon2, polygon3]

# # # Get the unified polygon
# # unified_polygon = unify_polygons(polygons)

In [12]:
def muliple_area_buffer(polygon, area_multiple, precision=100):
    """
    This function takes a polygon and returns a polygon with a buffer such that the area of the buffered polygon
    is approximately the specified multiple the area of the original polygon.

    :param polygon: Shapely Polygon object
    :param area_multiple: the multiple of the original polygon area you wish the buffered polygon to be
    :param precision: Precision for the iterative process to find the buffer distance
    :return: Buffered Polygon
    """
    original_area = polygon.area
    target_area = area_multiple * original_area
    buffer_distance = 0
    buffered_polygon = polygon

    while True:
        buffered_polygon = polygon.buffer(buffer_distance)
        if buffered_polygon.area >= target_area:
            break
        buffer_distance += precision
    
    # Convert to geodataframe
    buffered_polygon_gdf = gpd.GeoDataFrame({'geometry': [buffered_polygon]})
    
    # Set CRS
    buffered_polygon_gdf.geometry.crs = 'EPSG:3031'

    return buffered_polygon_gdf

# # Example usage
# # Define a simple square polygon
# square = Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])

# # Apply the function to find the buffered polygon area and bounds
# buffered_poly = double_area_buffer(square)
# buffered_poly.area, buffered_poly.bounds

In [13]:
def create_folder(directory):
    # Check if the directory already exists
    if not os.path.exists(directory):
        # If it doesn't exist, create a new directory
        os.makedirs(directory)
        print(f"Folder '{directory}' created successfully.")
    else:
        print(f"Folder '{directory}' already exists.")

In [14]:
def data_counts(lake_gdf, dataset1, dataset2): 
    '''
    Create planview plots of counts going into gridded ice-surface height change (dh/dt) products
    
    Inputs:
    * lake_gdf: geodataframe of lake to be analyzed
    * dataset1: dataset1 to be analyzed
    * dataset2: dataset2 to be analyzed in conjunction with dataset1; currently configured to 
    splice CryoSat-2 and ICESat-2 eras
    
    Outputs: 
    * Sequence of planview data count visuals of lake around surrounding area
    '''
    # Define lake name and polygon
    lake_name = lake_gdf.name.values[0]
    lake_poly = lake_gdf.iloc[0].geometry
    
    # Create buffered polygons for various multiples of lake area to find which
    # best emcompasses the height change signals at previously identified lakes
    buffered_poly_2x = muliple_area_buffer(lake_poly, 2)
    buffered_poly_3x = muliple_area_buffer(lake_poly, 3)
    buffered_poly_4x = muliple_area_buffer(lake_poly, 4)
    buffered_poly_5x = muliple_area_buffer(lake_poly, 5)

    # Clipping datasets
    dataset1_clipped = dataset1.rio.clip(buffered_poly_5x.geometry, dataset1.rio.crs)
    dataset2_clipped = dataset2.rio.clip(buffered_poly_5x.geometry, dataset2.rio.crs)

    # Establish x_min, x_max, y_min, y_max
    x_min, y_min, x_max, y_max = buffered_poly_5x.iloc[0].geometry.bounds
    x_buffer = abs(x_max-x_min)*0.2
    y_buffer = abs(y_max-y_min)*0.2
   
    # Make output folders
    create_folder(OUTPUT_DIR + '/{}'.format('data_counts'))
    create_folder(OUTPUT_DIR + '/data_counts/{}'.format(lake_name))
    print('Now creating/saving plots...')

    # Create lines for legend
    S09_color = 'cyan'
    SF18_color  = 'darkcyan'
    S24_color = 'deepskyblue'
    Smith2009 = plt.Line2D((0, 1), (0, 0), color=S09_color, linestyle=(0, (1, 5)), linewidth=2)
    SiegfriedFricker2018 = plt.Line2D((0, 1), (0, 0), color=SF18_color, linestyle=(0, (1, 3)), linewidth=2)
    Sauthoff2024 = plt.Line2D((0, 1), (0, 0), color=S24_color, linestyle=(0, (1, 1)), linewidth=2)

    # Calculate cycle-to-cycle dHeight at each cycle of the spliced data sets
    for idx in range(len(midcyc_dates)):
        # For midcyc_dates indexes <= 32, use CryoSat-2 dataset for cycle-to-cycle dHeight
        # This covers the CryoSat-2 era before ICESat-2 launch (2010-08-17 to 2018-08-17)
        if idx <= 32:
            count_clipped = dataset1_clipped['data_count'][idx+1,:,:]-dataset1_clipped['data_count'][idx,:,:]
            count_clipped.rio.write_crs("epsg:3031", inplace=True)

        # For midcyc_dates indexes > 32, use ICESat-2 dataset for cycle-to-cycle dHeight
        # This covers the ICESat-2 era (2018-11-16 to most recently available data)
        elif idx > 32:
            # Subtract 32 from idx to start over with new dataset
            count_clipped = dataset2_clipped['data_count'][(idx-33)+1,:,:]-dataset2_clipped['data_count'][idx-33,:,:]
            count_clipped.rio.write_crs("epsg:3031", inplace=True)
          
        # Only plot arrays that have data in them; some time slices have no data because no CryoSat-2 SARIn coverage
        # Returns True if there is at least one non-NaN value in data_stacked, and False if all values are NaN
        if np.any(~np.isnan(count_clipped)):
            # Create fig, ax
            fig, ax = plt.subplots()

            # Plot figure
            img = ax.imshow(count_clipped, extent=[x_min, x_max, y_min, y_max], 
                origin='lower', cmap='viridis')

            # Plot buffered polygons
            buffered_poly_2x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            buffered_poly_3x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            buffered_poly_4x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            buffered_poly_5x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            
            # Overlay published active lake outlines for visual comparison and grounding line
            # Smith2009_outlines.boundary.plot(ax=ax, facecolor=S09_color, linestyle=(0, (1, 10)), linewidth=2, alpha=0.25)
            Smith2009_outlines.boundary.plot(ax=ax, edgecolor=S09_color, linestyle=(0, (1, 5)), linewidth=2)
            # SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, facecolor=SF18_color, linestyle=(0, (1, 5)), linewidth=2, alpha=0.25)
            SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, edgecolor=SF18_color, linestyle=(0, (1, 3)), linewidth=2)
            # Sauthoff2023_S23outlines.boundary.plot(ax=ax, facecolor=S23_color, linestyle=(0, (1, 1)), linewidth=2, alpha=0.25)
            Sauthoff2024_S24outlines.plot(ax=ax, edgecolor=S24_color, linestyle=(0, (1, 1)), linewidth=2)          
            
            # Change polar stereographic m to km
            km_scale = 1e3
            ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
            ax.xaxis.set_major_formatter(ticks_x)
            ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
            ax.yaxis.set_major_formatter(ticks_y)  

            # Label axes, set limits, and set title
            ax.set_xlabel('x [km]', size=15)
            ax.set_ylabel('y [km]', size=15) 
            ax.set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

            # Plot inset map
            axIns = ax.inset_axes([-0.01, 0.01, 0.3, 0.3]) # [left, bottom, width, height] (fractional axes coordinates)
            axIns.set_aspect('equal')
            moa_2014_coastline.plot(ax=axIns, color='gray', edgecolor='k', linewidth=0.1, zorder=3)
            moa_2014_groundingline.plot(ax=axIns, color='ghostwhite', edgecolor='k', linewidth=0.1, zorder=3)
            axIns.axis('off')

            # Plot red star to indicate location
            axIns.scatter(((x_max+x_min)/2), ((y_max+y_min)/2), marker='*', 
                linewidth=1, edgecolor='k', facecolor='r', s=100, zorder=3)

            # Add colorbar 
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.2)
            fig.colorbar(img, cax=cax).set_label('data counts', size=15)

            # Add legend
            ax.legend([Smith2009, 
                       SiegfriedFricker2018, 
                       Sauthoff2024
                      ],
                ['static outline (S09)', 
                 'static outline (SF18)', 
                 'static point (S24)'
                ], 
                loc='upper center')
            
            # Set a title for the axes
            ax.set_title('Data counts from from {} to {}'.format(cyc_start_dates[idx].astype('datetime64[D]').astype(str), cyc_end_dates[idx].astype('datetime64[D]').astype(str)))
            
            # Save and close fig
            plt.savefig(OUTPUT_DIR + 
                '/data_counts/{}/data_counts_{}_{}-{}.png'
                .format(lake_name, lake_name, cyc_start_dates[idx].astype('datetime64[D]').astype(str), cyc_end_dates[idx].astype('datetime64[D]').astype(str)), dpi=300, bbox_inches='tight')
            plt.close()
    print('Complete')
        
# # Example usage
# data_counts(lake_gdf=lake_gdf, dataset1=CS2_Smith2017, dataset2=ATL15_dh): 

In [15]:
def height_change_time_series(lake_gdf, dataset1, dataset2): 
    '''
    Create planview plots of ice surface height changes (dh/dt)
    
    Inputs:
    * lake_gdf: geodataframe of lake to be analyzed
    * dataset1: dataset1 to be analyzed
    * dataset2: dataset2 to be analyzed in conjunction with dataset1; currently configured to 
    splice CryoSat-2 and ICESat-2 eras
    
    Outputs: 
    * Sequence of planview delta height visuals of lake around surrounding area
    '''
    # Define lake name and polygon
    lake_name = lake_gdf.name.values[0]
    lake_poly = lake_gdf.iloc[0].geometry
    
    # Create buffered polygons for various multiples of lake area to find which
    # best emcompasses the height change signals at previously identified lakes
    buffered_poly_2x = muliple_area_buffer(lake_poly, 2)
    buffered_poly_3x = muliple_area_buffer(lake_poly, 3)
    buffered_poly_4x = muliple_area_buffer(lake_poly, 4)
    buffered_poly_5x = muliple_area_buffer(lake_poly, 5)

    # Clipping datasets
    dataset1_clipped = dataset1.rio.clip(buffered_poly_5x.geometry, dataset1.rio.crs)
    dataset2_clipped = dataset2.rio.clip(buffered_poly_5x.geometry, dataset2.rio.crs)

    # Establish x_min, x_max, y_min, y_max
    x_min, y_min, x_max, y_max = buffered_poly_5x.iloc[0].geometry.bounds
    x_buffer = abs(x_max-x_min)*0.2
    y_buffer = abs(y_max-y_min)*0.2
   
    # Make output folders
    create_folder(OUTPUT_DIR + '/{}'.format('height_change_time_series'))
    create_folder(OUTPUT_DIR + '/height_change_time_series/{}'.format(lake_name))
    print('Now creating/saving plots...')

    # Create lines for legend
    S09_color = 'cyan'
    SF18_color  = 'darkcyan'
    S24_color = 'deepskyblue'
    Smith2009 = plt.Line2D((0, 1), (0, 0), color=S09_color, linestyle=(0, (1, 5)), linewidth=2)
    SiegfriedFricker2018 = plt.Line2D((0, 1), (0, 0), color=SF18_color, linestyle=(0, (1, 3)), linewidth=2)
    Sauthoff2024 = plt.Line2D((0, 1), (0, 0), color=S24_color, linestyle=(0, (1, 1)), linewidth=2)

    # Calculate cycle-to-cycle dHeight at each cycle of the spliced data sets
    for idx in range(len(midcyc_dates)):
        # For midcyc_dates indexes <= 32, use CryoSat-2 dataset for cycle-to-cycle dHeight
        # This covers the CryoSat-2 era before ICESat-2 launch (2010-08-17 to 2018-08-17)
        if idx <= 32:
            dhdt_clipped = dataset1_clipped.delta_h[idx+1,:,:]-dataset1_clipped.delta_h[idx,:,:]
            dhdt_clipped.rio.write_crs("epsg:3031", inplace=True)

        # For midcyc_dates indexes > 32, use ICESat-2 dataset for cycle-to-cycle dHeight
        # This covers the ICESat-2 era (2018-11-16 to most recently available data)
        elif idx > 32:
            # Subtract 32 from idx to start over with new dataset
            dhdt_clipped = dataset2_clipped.delta_h[(idx-33)+1,:,:]-dataset2_clipped.delta_h[idx-33,:,:]
            dhdt_clipped.rio.write_crs("epsg:3031", inplace=True)
          
        # Only plot arrays that have data in them; some time slices have no data because no CryoSat-2 SARIn coverage
        # Returns True if there is at least one non-NaN value in data_stacked, and False if all values are NaN
        if np.any(~np.isnan(dhdt_clipped)):
            # Create fig, ax
            fig, ax = plt.subplots()

            # Plot figure
            img = ax.imshow(dhdt_clipped, extent=[x_min, x_max, y_min, y_max], 
                origin='lower', cmap='coolwarm_r', 
                norm=colors.CenteredNorm())

            # Plot buffered polygons
            buffered_poly_2x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            buffered_poly_3x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            buffered_poly_4x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            buffered_poly_5x.boundary.plot(ax=ax, edgecolor='k', facecolor='none', linewidth=0.5)
            
            # Overlay published active lake outlines for visual comparison and grounding line
            # Smith2009_outlines.boundary.plot(ax=ax, facecolor=S09_color, linestyle=(0, (1, 10)), linewidth=2, alpha=0.25)
            Smith2009_outlines.boundary.plot(ax=ax, edgecolor=S09_color, linestyle=(0, (1, 5)), linewidth=2)
            # SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, facecolor=SF18_color, linestyle=(0, (1, 5)), linewidth=2, alpha=0.25)
            SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, edgecolor=SF18_color, linestyle=(0, (1, 3)), linewidth=2)
            # Sauthoff2023_S23outlines.boundary.plot(ax=ax, facecolor=S23_color, linestyle=(0, (1, 1)), linewidth=2, alpha=0.25)
            Sauthoff2024_S24outlines.plot(ax=ax, edgecolor=S24_color, linestyle=(0, (1, 1)), linewidth=2)          
            
            # Change polar stereographic m to km
            km_scale = 1e3
            ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
            ax.xaxis.set_major_formatter(ticks_x)
            ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
            ax.yaxis.set_major_formatter(ticks_y)  

            # Label axes, set limits, and set title
            ax.set_xlabel('x [km]', size=15)
            ax.set_ylabel('y [km]', size=15) 
            ax.set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

            # Plot inset map
            axIns = ax.inset_axes([-0.01, 0.01, 0.3, 0.3]) # [left, bottom, width, height] (fractional axes coordinates)
            axIns.set_aspect('equal')
            moa_2014_coastline.plot(ax=axIns, color='gray', edgecolor='k', linewidth=0.1, zorder=3)
            moa_2014_groundingline.plot(ax=axIns, color='ghostwhite', edgecolor='k', linewidth=0.1, zorder=3)
            axIns.axis('off')

            # Plot red star to indicate location
            axIns.scatter(((x_max+x_min)/2), ((y_max+y_min)/2), marker='*', 
                linewidth=1, edgecolor='k', facecolor='r', s=100, zorder=3)

            # Add colorbar 
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.2)
            fig.colorbar(img, cax=cax).set_label('height change (dh) [m]', size=15)

            # Add legend
            ax.legend([Smith2009, 
                       SiegfriedFricker2018, 
                       Sauthoff2024
                      ],
                ['static outline (S09)', 
                 'static outline (SF18)', 
                 'static point (S24)'
                ], 
                loc='upper center')
            
            # Set a title for the axes
            ax.set_title('Height change from from {} to {}'.format(cyc_start_dates[idx].astype('datetime64[D]').astype(str), cyc_end_dates[idx].astype('datetime64[D]').astype(str)))
            
            # Save and close fig
            plt.savefig(OUTPUT_DIR + 
                '/height_change_time_series/{}/height_change_time_series_{}_{}-{}.png'
                .format(lake_name, lake_name, cyc_start_dates[idx].astype('datetime64[D]').astype(str), cyc_end_dates[idx].astype('datetime64[D]').astype(str)), dpi=300, bbox_inches='tight')
            plt.close()
    print('Complete')
        
# # Example usage
# find_evolving_outlines(lake_gdf=lake_gdf, dataset1=CS2_Smith2017, dataset2=ATL15_dh): 

In [16]:
# Create gdf for static outline geometric variables
    # Create empty list to store polygons, areas, perimeters and dates
    # static_areas = []
    # static_dhs = []
    # static_dvols = []
    
    
    # # Store static outline area, dh, and dvol
    # dhdt_poly = dhdt_clipped.rio.clip([lake_gdf.geometry.values[0]])
    # poly_area = lake_gdf.geometry.area[0]
    # static_areas += [poly_area]
    # poly_dh = np.nanmean(dhdt_poly)
    # static_dhs += [poly_dh]
    # poly_dvol = poly_dh*poly_area
    # static_dvols += [lake_gdf.geometry.area[0]]

In [118]:
def find_evolving_outlines(lake_gdf, area_multiple, level, dataset1, dataset2): 
    '''
    Create planview dh/dt plots of ice surface height changes 
    Create time-variable outlines using skimage contour to plot evolving outlines as polygons.
    
    Inputs:
    * ROI_name: str of the region of interest for using in file name saving
    * ROI_poly: list of polygon(s) that define evolving outline search extent
    * level: vertical distance in meters to delineate ice surface deformation contour
    * dataset1: dataset1 to be analyzed
    * dataset2: dataset2 to be analyzed in conjunction with dataset1; currently configured to 
    splice CryoSat-2 and ICESat-2 eras
    
    Outputs: 
    * Sequence of planview delta height visuals of ICESat-2 ATL15 with variable ice surface 
    deformation contours plotted to delineate evolving lake boundaries.
    * geopandas geodataframe of polygons created at each step (would need to modify to collect all polygons 
    at all time steps)
    '''
    # Define lake name and polygon
    lake_name = lake_gdf.name.values[0]
    lake_poly = lake_gdf.iloc[0].geometry
    buffered_gdf = muliple_area_buffer(lake_poly, area_multiple)
    
    # Clipping datasets
    if dataset1 != 'none':
        dataset1_clipped = dataset1.rio.clip(buffered_gdf.geometry, dataset1.rio.crs)
    dataset2_clipped = dataset2.rio.clip(buffered_gdf.geometry, dataset2.rio.crs)
    
    # Establish x_min, x_max, y_min, y_max
    x_min, y_min, x_max, y_max = buffered_gdf.iloc[0].geometry.bounds
    x_buffer = abs(x_max-x_min)*0.2
    y_buffer = abs(y_max-y_min)*0.2
    
    # Make output folders
    create_folder(OUTPUT_DIR + '/{}'.format('find_evolving_outlines'))
    create_folder(OUTPUT_DIR + '/find_evolving_outlines/{}'.format(lake_name))
    print('Now creating/saving plots...')
    
    # Create lines for legend
    S09_color = 'cyan'
    SF18_color  = 'darkcyan'
    S24_color = 'deepskyblue'
    Smith2009 = plt.Line2D((0, 1), (0, 0), color=S09_color, linestyle=(0, (1, 5)), linewidth=2)
    SiegfriedFricker2018 = plt.Line2D((0, 1), (0, 0), color=SF18_color, linestyle=(0, (1, 3)), linewidth=2)
    Sauthoff2024 = plt.Line2D((0, 1), (0, 0), color=S24_color, linestyle=(0, (1, 1)), linewidth=2)
    uplift = plt.Line2D((0, 1), (0, 0), color='mediumblue', linestyle='dashdot', linewidth=2)
    subsidence = plt.Line2D((0, 1), (0, 0), color='maroon', linestyle=(0, (3, 1, 1, 1)), linewidth=2)
    
    # Create empty list to store polygons, areas, dh's, dvol's and dates
    polys = []
    areas = []
    dhs = []
    dvols =[]
    midcyc_datetimes = []

    # Calculate cycle-to-cycle dHeight at each cycle of the spliced data sets
    for idx in range(len(midcyc_dates)-1):
        # For midcyc_dates indexes <= 32, use CryoSat-2 dataset for cycle-to-cycle dHeight
        # This covers the CryoSat-2 era before ICESat-2 launch (2010-08-17 to 2018-08-17)
        if idx <= 32:
            if dataset1 == 'none':
                continue
            else:
                dhdt_clipped = dataset1_clipped.delta_h[idx+1,:,:]-dataset1_clipped.delta_h[idx,:,:]
                dhdt_clipped.rio.write_crs("epsg:3031", inplace=True)
        # For midcyc_dates indexes > 32, use ICESat-2 dataset for cycle-to-cycle dHeight
        # This covers the ICESat-2 era (2018-11-16 to most recently available data)
        elif idx > 32:
            # Subtract 33 (32th idx because 0th idx) from idx to start over with new dataset
            dhdt_clipped = dataset2_clipped.delta_h[(idx+1)-32,:,:]-dataset2_clipped.delta_h[idx-32,:,:]
            dhdt_clipped.rio.write_crs("epsg:3031", inplace=True)

        # Create mapping conversion factor to map array location to polar stereographic x,y
        x_conv = (x_max-x_min)/dhdt_clipped.shape[1]
        y_conv = (y_max-y_min)/dhdt_clipped.shape[0]

        # Only plot arrays that have data in them; some time slices have no data because no CryoSat-2 SARIn coverage
        # Returns True if there is at least one non-NaN value in data_stacked, and False if all values are NaN
        if np.any(~np.isnan(dhdt_clipped)):
            # Create fig, ax
            fig, ax = plt.subplots()

            # Plot figure
            img = ax.imshow(dhdt_clipped, extent=[x_min, x_max, y_min, y_max], 
                origin='lower', cmap='coolwarm_r', 
                norm=colors.CenteredNorm())

            # Plot the boundary of the evolving outline search extent
            gpd.GeoSeries(buffered_gdf.iloc[0].geometry).boundary.plot(ax=ax, color='red')

            # Create empty lists to store contours 
            contours_pos = []
            contours_neg = []

            # Create contours at the positive and negative threshold 
            contour_pos = measure.find_contours(dhdt_clipped.values, level)
            # If at least one contour, add to list of contours
            if len(contour_pos) > 0: 
                contours_pos += [contour_pos]
            contour_neg = measure.find_contours(dhdt_clipped.values, -level)
            if len(contour_neg) > 0: 
                contours_neg += [contour_neg]

            # Plot contours and make into polygons
            for i in range(len(contours_pos)): 
                for j in range(len(contours_pos[i])):
                    x = x_min+contours_pos[i][j][:,1]*x_conv
                    y = y_min+contours_pos[i][j][:,0]*y_conv
                    ax.plot(x, y, color='mediumblue', linestyle='dashdot', linewidth=1, label=level)

                    # Make polygons from evolving outlines and store to list
                    if len(contours_pos[i][j][:,1]) > 2: 
                        poly = Polygon(list(zip(x, y)))
                        try:
                                dhdt_poly = dhdt_clipped.rio.clip([poly])
                                polys += [poly]
                                lon, lat = XY_TO_LL.transform(x,y)
                                poly_area = abs(GEOD.polygon_area_perimeter(lon, lat)[0])
                                areas += [poly_area]
                                poly_dh = np.nanmean(dhdt_poly)
                                dhs += [poly_dh]
                                poly_dvol = poly_dh*poly_area
                                dvols += [poly_dvol]
                                midcyc_datetimes += [midcyc_dates[idx]]
                        except NoDataInBounds:
                            continue  # Skip to the next iteration
                        except Exception as e:
                            # Handle any other exceptions
                            raise

            for i in range(len(contours_neg)): 
                for j in range(len(contours_neg[i])):
                    x = x_min+contours_neg[i][j][:,1]*x_conv
                    y = y_min+contours_neg[i][j][:,0]*y_conv
                    ax.plot(x, y, color='maroon', linestyle=(0, (3, 1, 1, 1)), linewidth=1, label=-level)

                    if len(contours_neg[i][j][:,1]) > 2: 
                        poly = Polygon(list(zip(x, y)))
                        try:
                                dhdt_poly = dhdt_clipped.rio.clip([poly])
                                polys += [poly]
                                lon, lat = XY_TO_LL.transform(x,y)
                                poly_area = abs(GEOD.polygon_area_perimeter(lon, lat)[0])
                                areas += [poly_area]
                                # poly_perim = abs(GEOD.polygon_area_perimeter(lon, lat)[1])
                                # perims += [poly_perim]
                                poly_dh = np.nanmean(dhdt_poly)
                                dhs += [poly_dh]
                                poly_dvol = poly_dh*poly_area
                                dvols += [poly_dvol]
                                midcyc_datetimes += [midcyc_dates[idx]]
                        except NoDataInBounds:
                            continue  # Skip to the next iteration
                        except Exception as e:
                            # Handle any other exceptions
                            raise

            # Overlay published active lake outlines for visual comparison and grounding line
            S09_color = 'cyan'
            SF18_color  = 'darkcyan'
            S24_color = 'deepskyblue'
            # Smith2009_outlines.boundary.plot(ax=ax, facecolor=S09_color, linestyle=(0, (1, 10)), linewidth=2, alpha=0.25)
            Smith2009_outlines.boundary.plot(ax=ax, edgecolor=S09_color, linestyle=(0, (1, 10)), linewidth=2)
            # SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, facecolor=SF18_color, linestyle=(0, (1, 5)), linewidth=2, alpha=0.25)
            SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, edgecolor=SF18_color, linestyle=(0, (1, 5)), linewidth=2)
            # Sauthoff2023_S23outlines.boundary.plot(ax=ax, facecolor=S23_color, linestyle=(0, (1, 1)), linewidth=2, alpha=0.25)
            Sauthoff2024_S24outlines.boundary.plot(ax=ax, edgecolor=S24_color, linestyle=(0, (1, 1)), linewidth=2)

            # Change polar stereographic m to km
            km_scale = 1e3
            ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
            ax.xaxis.set_major_formatter(ticks_x)
            ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
            ax.yaxis.set_major_formatter(ticks_y)  

            # Label axes, set limits, and set title
            ax.set_xlabel('x [km]', size=15)
            ax.set_ylabel('y [km]', size=15) 
            ax.set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

            # Plot inset map
            axIns = ax.inset_axes([-0.01, 0.01, 0.3, 0.3]) # [left, bottom, width, height] (fractional axes coordinates)
            axIns.set_aspect('equal')
            moa_2014_coastline.plot(ax=axIns, color='gray', edgecolor='k', linewidth=0.1, zorder=3)
            moa_2014_groundingline.plot(ax=axIns, color='ghostwhite', edgecolor='k', linewidth=0.1, zorder=3)
            axIns.axis('off')

            # # Plot black rectangle to indicate location
            # rect = Rectangle((x_min, y_min), (x_max-x_min), (y_max-y_min), fill=False, linewidth=2, color='k', zorder=3)
            # axIns.add_artist(rect)

            # Plot red star to indicate location
            axIns.scatter(((x_max+x_min)/2), ((y_max+y_min)/2), marker='*', 
                linewidth=1, edgecolor='k', facecolor='r', s=100, zorder=3)

            # Add colorbar 
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.2)
            fig.colorbar(img, cax=cax).set_label('height change (dh) [m]', size=15)
            
            # Add legend
            ax.legend([Smith2009, 
                       SiegfriedFricker2018, 
                       Sauthoff2024,
                       uplift, 
                       subsidence],
                ['static outline (S09)', 
                 'static outline (SF18)', 
                 'static point (S24)',
                 ('+ '+str(level)+' m uplift evolving outline'), 
                 ('– '+str(level)+' m subsidence evolving outline')], 
                loc='upper left')
            
            # Set a title for the axes
            ax.set_title('Evolving outlines and height change \nfrom from {} to {}'.format(cyc_start_dates[idx].astype('datetime64[D]').astype(str), cyc_end_dates[idx].astype('datetime64[D]').astype(str)))
            
            # Save and close fig
            plt.savefig(OUTPUT_DIR + 
                '/find_evolving_outlines/{}/find_evolving_outlines_{}_{}m-level_{}-{}.png'
                .format(lake_name, lake_name, level, cyc_start_dates[idx].astype('datetime64[D]').astype(str), cyc_end_dates[idx].astype('datetime64[D]').astype(str)), dpi=300, bbox_inches='tight')
            plt.close()
    
    # Store polygons in geopandas geodataframe for further analysis
    levels = [level for _ in range(len(polys))]
    d = {'level': levels,
         'geometry': polys, 
         'area (m^2)': areas, 
         'dh (m)': dhs, 
         'vol (m^3)': dvols,
         'midcyc_datetime': midcyc_datetimes}
    gdf = gpd.GeoDataFrame(d, crs="EPSG:3031")
    gdf['centroid'] = gdf['geometry'].centroid

    print('Complete')

    return gdf

# # Example usage
# gdf = find_evolving_outlines(ROI, threshold, dataset1, dataset2)

In [18]:
def plot_evolving_outlines(gdf):
    '''
    Func to plot evolving outlines in aggregate
    '''
    # Create fig, ax
    fig, ax = plt.subplots(figsize=(5,5))

    # Set colormap and normalize to date values
    cmap = plt.get_cmap('plasma', len(midcyc_dates)-1)
    norm = plt.Normalize(datetime64_to_fractional_year(midcyc_dates[0]), datetime64_to_fractional_year(midcyc_dates[-1]))
    m = plt.cm.ScalarMappable(cmap=cmap)
    m.set_array(np.array([datetime64_to_fractional_year(date) for date in midcyc_dates[0:-1]]))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="2.5%", pad=0.5)
    fig.colorbar(m, cax=cax, orientation='horizontal').set_label('evolving outline year', size=10)

    # Overlay published active lake outlines for visual comparison and grounding line
    S09_color = 'cyan'
    SF18_color  = 'darkcyan'
    S23_color = 'deepskyblue'
    Smith2009_outlines.boundary.plot(ax=ax, facecolor=S09_color, linestyle=(0, (1, 5)), linewidth=2, alpha=0.25)
    Smith2009_outlines.boundary.plot(ax=ax, edgecolor=S09_color, linestyle=(0, (1, 5)), linewidth=2)
    SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, facecolor=SF18_color, linestyle=(0, (1, 3)), linewidth=2, alpha=0.25)
    SiegfriedFricker2018_SF18outlines.boundary.plot(ax=ax, edgecolor=SF18_color, linestyle=(0, (1, 3)), linewidth=2)
    Sauthoff2024_S24outlines.boundary.plot(ax=ax, facecolor=S23_color, linestyle=(0, (1, 1)), linewidth=2, alpha=0.25)
    Sauthoff2024_S24outlines.boundary.plot(ax=ax, edgecolor=S23_color, linestyle=(0, (1, 1)), linewidth=2)

    # Use for loop to plot each outline in the geopandas dataframe and color by date
    lines = []  # list of lines to be used for the legend
    for idx, dt in enumerate(midcyc_dates, 0):
        x = 1; y = 1
        line, = ax.plot(x, y, color=cmap(norm(datetime64_to_fractional_year(midcyc_dates[idx]))), linewidth=5)
        lines.append(line)
        if any(gdf[gdf['midcyc_datetime'] == dt]):
            gdf[gdf['midcyc_datetime'] == dt].boundary.plot(ax=ax, 
                color=cmap(norm(datetime64_to_fractional_year(midcyc_dates[idx])))
                )

    # Change polar stereographic m to km for cleaner-looking axes labels
    km_scale = 1e3
    ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
    ax.xaxis.set_major_formatter(ticks_x)
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
    ax.yaxis.set_major_formatter(ticks_y)
    ax.set_aspect('equal')

    # Label axes and set limits
    ax.set_xlabel('x [km]', size=10)
    ax.set_ylabel('y [km]', size=10)
    # Extract x_min, y_min, x_max, y_max from the total_bounds attribute
    x_min, y_min, x_max, y_max = gdf.total_bounds
    x_buffer = abs(x_max-x_min)*0.2
    y_buffer = abs(y_max-y_min)*0.2
    ax.set(xlim=(x_min-x_buffer, x_max+x_buffer), ylim=(y_min-y_buffer, y_max+y_buffer))

    # Create lines for legend
    Smith2009 = plt.Line2D((0, 1), (0, 0), color=S09_color, linestyle=(0, (1, 5)), linewidth=5)
    SiegfriedFricker2018 = plt.Line2D((0, 1), (0, 0), color=SF18_color, linestyle=(0, (1, 3)), linewidth=5)
    
    # Create legend
    ax.legend(handles=[Smith2009, 
        SiegfriedFricker2018, 
        tuple(lines)], 
        labels=['static outline (Smith and others, 2009)',
            'static outline (Siegfried & Fricker, 2018)', 
            # 'evolving outline ({} m threshold)'.format(threshold)], 
            'evolving outline (this study)'], 
              handlelength=3, handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
             loc='upper left', bbox_to_anchor=(0, 1.2))

    # Plot inset map to show location 
    axIns = ax.inset_axes([0.02, 0.01, 0.25, 0.25]) # [left, bottom, width, height] (fractional axes coordinates)
    axIns.set_aspect('equal')
    moa_2014_coastline.plot(ax=axIns, color='gray', edgecolor='k', linewidth=0.1, zorder=3)
    moa_2014_groundingline.plot(ax=axIns, color='ghostwhite', edgecolor='k', linewidth=0.1, zorder=3)
    axIns.scatter(((x_max+x_min)/2), ((y_max+y_min)/2), marker='*', linewidth=1, edgecolor='k', facecolor='r', s=50, zorder=3)
    # rect = Rectangle((x_min, y_min), (x_max-x_min), (y_max-y_min), fill=False, linewidth=2, color='k', zorder=3)
    # axIns.add_artist(rect) 
    axIns.axis('off')

    # plt.savefig(OUTPUT_DIR + '/plot_evolving_outlines_time_series/{}/plot_evolving_outlines_time_series_{}_{}km-buffer_{}m-threshold-{}outlines.png'
    #     .format(ROI, ROI, buffer/1e3, threshold, len(gdf)),
    #             dpi=300, bbox_inches='tight')

    plt.show()

In [55]:
def extract_intersecting_polygons(gdf, polygon):
    """
    Extract polygons from a GeoDataFrame that intersect with a specified polygon,
    as well as polygons that intersect with those intersected polygons.

    Parameters:
    gdf (geopandas.GeoDataFrame): A GeoDataFrame containing polygon geometries.
    polygon (shapely.Polygon): A shapely polygon

    Returns:
    geopandas.GeoDataFrame: A GeoDataFrame containing the polygons that either
                            intersect directly with the input polygon or intersect with
                            polygons that intersect with the initial polygon.
    """
    # Removing direct point geometry creation and using the 'polygon' directly

    # Step 1: Find polygons that intersect with the given polygon
    intersecting_polygons = gdf[gdf.intersects(polygon)]

    # Step 2: Coverage of the approach to look for intersections with the resulting polygon set
    result_polygons = gdf[gdf.geometry.intersects(intersecting_polygons.unary_union)]

    return result_polygons

# The proposed `polygon` parameter should be an object of shapely.geometry.Polygon.
# Like the point, the polygon should be provided explicitly.

# New example usage, assuming a more considerable area of interest than a simple coordinate point:
# from shapely.geometry import Polygon
# gdf = geopandas.read_file("your_file.geojson")
# Example polygon creation could be like:
# polygon = Polygon([(-104.05, 48.99), (-97.22,  48.98), (-96.58,  45.94), (-104.03, 45.94), (-104.05, 48.99)])
# result = extract_intersecting_polygons_v2(gdf, polygon)

In [20]:
# def remove_outlier_polygon(gdf, axis='x', max_or_min='max'):
#     '''
#     Func to remove outline outliers by removing a polygon
#     with the most extreme x or y coordinates
    
#     Inputs
#     * geopandas geodataframe
#     * axis indicates whether the x or y axis will 
#     have its extreme polygon removed
#     * max_or_min indicates whether the max or min centroid axis
#     value will be removed
    
#     Ouputs
#     * geopandas geodataframe with one outline removed
#     '''
    
#     # Check if the GeoDataFrame is empty
#     if gdf.empty:
#         print("GeoDataFrame is empty. Nothing to remove.")
#         return gdf
    
#     # Choose the axis and max or min for extreme value
#     if axis not in ['x', 'y']:
#         raise ValueError("Invalid axis. Use 'x' or 'y'.")
#     if max_or_min not in ['max', 'min']:
#         raise ValueError("Invalid max_or_min. Use 'max' or 'min'.")

#     # Calculate the centroid and extreme value
#     if axis == 'x':
#         if max_or_min == 'max':
#             idx_to_remove = gdf.geometry.bounds['maxx'].idxmax()
#         else:
#             idx_to_remove = gdf.geometry.bounds['minx'].idxmin()
#     else:
#         if max_or_min == 'max':
#             idx_to_remove = gdf.geometry.bounds['maxy'].idxmax()
#         else:
#             idx_to_remove = gdf.geometry.bounds['miny'].idxmin()

#     # Remove the polygon with the specified polygon index to remove
#     gdf_filtered = gdf.drop(idx_to_remove)

#     print(f"Removed polygon with extreme {max_or_min} {axis}-value at index {idx_to_remove}.")

#     return gdf_filtered

# # Example usage:
# # Assuming 'gdf' is your GeoDataFrame
# # new_gdf = remove_extreme_polygon(gdf, axis='x', positive=True)

In [21]:
# Func to plot aggregate evolving outlines


# Import datasets

In [22]:
# Import subglacial lake outlines 
exec(open(SCRIPT_DIR + '/Sauthoff2024_outlines.py').read())

In [23]:
# MODIS MOA 2014 coastline and grounding line
# https://nsidc.org/data/nsidc-0730/versions/1
shp = DATA_DIR + '/boundaries/MODIS_MOA/2014/moa2014_coastline_v01.shp' 
moa_2014_coastline = gpd.read_file(shp)
shp = DATA_DIR + '/boundaries/MODIS_MOA/2014/moa2014_grounding_line_v01.shp' 
moa_2014_groundingline = gpd.read_file(shp, crs=3031)
# moa_2014_groundingline['geometry'] = moa_2014_groundingline.buffer(0)

In [24]:
# Import Smith and others, 2017, TC method CryoSat-2 SARIn height and dheight data (closed source aquired from Ben Smith)
CS2_Smith2017 = xr.open_dataset(DATA_DIR + '/altimetry/CryoSat2/CS2_SARIn_Smith2017method/mos_2010.5_2019.0_relative_to_ATL14.nc')
CS2_Smith2017

<xarray.Dataset>
Dimensions:      (y: 4451, x: 5451, time: 34)
Coordinates:
  * y            (y) float64 -2.185e+06 -2.184e+06 ... 2.264e+06 2.265e+06
  * x            (x) float64 -2.665e+06 -2.664e+06 ... 2.784e+06 2.785e+06
  * time         (time) datetime64[ns] 2010-07-02T15:00:00 ... 2018-10-01T22:...
Data variables:
    mask         (y, x) float64 ...
    data_count   (time, y, x) float64 ...
    spatial_ref  int64 ...
    h            (time, y, x) float64 ...
    delta_h      (time, y, x) float64 ...
Attributes:
    fileName:                mos_2010.5_2021.5.h5
    shortName:               CS2-Smith-2017
    identifier_product_DOI:  doi:10.5194/tc-11-451-2017

In [25]:
# CS2_Smith2017_count = CS2_Smith2017.where(CS2_Smith2017['count'] > 0.5)

In [26]:
# Experimenting with ATL11 read in

In [27]:
# Log into NASA Earthdata to search for datasets
earthaccess.login()

In [28]:
# # Find ICESat-2 ATL11 r003 data granules
# results = earthaccess.search_data(
#     doi='10.5067/ATLAS/ATL11.006',
#     # short_name='ATL15',
#     # version='003',
#     bounding_box=(180, -90, -180, -60),  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
#     cloud_hosted=True,
# )

In [29]:
# # Open data granules as s3 files to stream
# files = earthaccess.open(results)
# # files

In [30]:
# files[1]

In [31]:
# import datatree as dt

# # Open the HDF5 file
# with h5py.File(files[0], 'r') as h5file:
#     # Load the entire HDF5 file into a DataTree
#     data_tree = dt.DataTree.from_hdf5(h5file)

# # Now `data_tree` is a DataTree object containing the structure and data of the HDF5 file
# # You can navigate and manipulate this tree structure as needed

# # For example, to print the contents of the DataTree
# print(data_tree)

In [32]:
# # create empty lists
# lon=[]
# lat=[]
# h_corr=[]
# sigma_h=[]

# # fill lists
# for file in files:
#     print(file)
#     try:
#         for pair in ['pt1','pt2','pt3']:
#             lons, lats, hh, ss=read_ATL11(file, pair)
#             lon += [lons]
#             lat += [lats]
#             h_corr += [hh]
#             sigma_h += [ss]
#     except Exception as E:
#         pass

# # concatenate lists
# lon=np.concatenate(lon)
# lat=np.concatenate(lat)
# h_corr=np.concatenate(h_corr, axis=0)
# sigma_h=np.concatenate(sigma_h, axis=0)
# x,y=ll2ps(lon,lat) # transform geodetic lon, lat to polar stereographic x, y

In [33]:
# Find ICESat-2 ATL15 r003 data granules
results = earthaccess.search_data(
    doi='10.5067/ATLAS/ATL15.003',
    # short_name='ATL15',
    # version='003',
    bounding_box=(180, -90, -180, -60),  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
    cloud_hosted=True,
)

Granules found: 16


In [34]:
# Open data granules as s3 files to stream
files = earthaccess.open(results)
files

Opening 16 granules, approx size: 5.05 GB


QUEUEING TASKS | :   0%|          | 0/16 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/16 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/16 [00:00<?, ?it/s]

[<File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A4_0318_40km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A4_0318_10km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A4_0318_20km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A2_0318_01km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A3_0318_20km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A2_0318_20km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A2_0318_40km_003_01.nc>,
 <File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A

In [35]:
# After viewing files, index the files you wish to open
print(files[15])
print(files[3])
print(files[9])
print(files[11])

<File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A1_0318_01km_003_01.nc>
<File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A2_0318_01km_003_01.nc>
<File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A3_0318_01km_003_01.nc>
<File-like object HTTPFileSystem, https://n5eil01u.ecs.nsidc.org/DP5/ATLAS/ATL15.003/2019.03.29/ATL15_A4_0318_01km_003_01.nc>


In [36]:
# Open each file, which are quadrants in polar stereographic coordinations around the Geographic South Pole
ATL15_A1 = xr.open_dataset(files[15], group='delta_h')
ATL15_A2 = xr.open_dataset(files[3], group='delta_h')
ATL15_A3 = xr.open_dataset(files[9], group='delta_h')
ATL15_A4 = xr.open_dataset(files[11], group='delta_h')

In [37]:
# # Open locally stored files when NSIDC cloud access isn't working
# ATL15_A1 = xr.open_dataset(DATA_DIR + '/altimetry/ICESat2/ATL15.003-Ant/ATL15_A1_0318_01km_003_01.nc', group='delta_h')
# ATL15_A2 = xr.open_dataset(DATA_DIR + '/altimetry/ICESat2/ATL15.003-Ant/ATL15_A2_0318_01km_003_01.nc', group='delta_h')
# ATL15_A3 = xr.open_dataset(DATA_DIR + '/altimetry/ICESat2/ATL15.003-Ant/ATL15_A3_0318_01km_003_01.nc', group='delta_h')
# ATL15_A4 = xr.open_dataset(DATA_DIR + '/altimetry/ICESat2/ATL15.003-Ant/ATL15_A4_0318_01km_003_01.nc', group='delta_h')

In [38]:
ATL15_A1

<xarray.Dataset>
Dimensions:              (x: 2631, y: 2271, time: 19)
Coordinates:
  * x                    (x) float64 0.0 1e+03 2e+03 ... 2.629e+06 2.63e+06
  * y                    (y) float64 0.0 1e+03 2e+03 ... 2.269e+06 2.27e+06
  * time                 (time) datetime64[ns] 2018-10-01T22:30:00 ... 2023-0...
Data variables:
    Polar_Stereographic  int8 ...
    ice_area             (time, y, x) float32 ...
    delta_h              (time, y, x) float32 ...
    delta_h_sigma        (time, y, x) float32 ...
    data_count           (time, y, x) float32 ...
    misfit_rms           (time, y, x) float32 ...
    misfit_scaled_rms    (time, y, x) float32 ...
Attributes:
    description:  delta_h group includes variables describing height differen...

In [39]:
# Specify the variables to keep
variables_to_keep = ['time', 'y', 'x', 'delta_h', 'data_count']

# List of xarray datasets
datasets = [ATL15_A1, ATL15_A2, ATL15_A3, ATL15_A4]

# Function to drop variables not in variables_to_keep from a dataset
def drop_unwanted_variables(dataset):
    variables_to_drop = [var for var in dataset.variables if var not in variables_to_keep]
    return dataset.drop_vars(variables_to_drop)

# Apply the function to each dataset
ATL15_A1, ATL15_A2, ATL15_A3, ATL15_A4 = [drop_unwanted_variables(ds) for ds in datasets]

In [40]:
# Use xarray concatenation to stitch two quadrants togethers
# Use xarray index selecting to occlude the duplicated x=0 vector of data
ATL15_A12 = xr.concat([ATL15_A2.isel(x=slice(0,-1)), ATL15_A1], dim="x")

In [41]:
# Use xarray concatenation to stitch two quadrants togethers
# Use xarray index selecting to occlude the duplicated x=0 vector of data
ATL15_A34 = xr.concat([ATL15_A3.isel(x=slice(0,-1)), ATL15_A4], dim='x')

In [42]:
# Use xarray concatenation to stitch two quadrants togethers
# Use xarray index selecting to occlude the duplicated x=0 vector of data
ATL15_dh = xr.concat([ATL15_A34.isel(y=slice(0,-1)), ATL15_A12], dim='y')

In [43]:
# Delete variables to reduce memory consumption
del ATL15_A1, ATL15_A12, ATL15_A2, ATL15_A3, ATL15_A34, ATL15_A4

In [44]:
# Add datasets attributes
ATL15_dh.attrs['identifier_product_DOI'] = '10.5067/ATLAS/ATL15.003'
ATL15_dh.attrs['shortName'] = 'ATL15'

In [45]:
ATL15_dh

<xarray.Dataset>
Dimensions:     (time: 19, x: 5461, y: 4461)
Coordinates:
  * time        (time) datetime64[ns] 2018-10-01T22:30:00 ... 2023-04-02T13:3...
  * x           (x) float64 -2.67e+06 -2.669e+06 ... 2.789e+06 2.79e+06
  * y           (y) float64 -2.19e+06 -2.189e+06 ... 2.269e+06 2.27e+06
Data variables:
    delta_h     (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    data_count  (time, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    description:             delta_h group includes variables describing heig...
    identifier_product_DOI:  10.5067/ATLAS/ATL15.003
    shortName:               ATL15

In [46]:
# Scripps Grounding Line
# https://doi.pangaea.de/10.1594/PANGAEA.819147
Scripps_gl = gpd.read_file(DATA_DIR + 
    '/boundaries/Depoorter2013/Antarctica_masks/scripps_antarctica_polygons_v1.shp')

# Isolate only land ice
Scripps_landice = Scripps_gl[Scripps_gl['Id_text'] == 'Grounded ice or land']

In [47]:
# Import MODIS Mosaic of Antarctica surface imagery
# https://nsidc.org/data/nsidc-0730/versions/1
# Relocate to data_dir
# Open into an xarray.DataArray
# moa_lowres = DATA_DIR + '/surface_imagery/MODIS_MOA/2014/moa750_2014_hp1_v01.tif' 
# moa_lowres_da = rioxarray.open_rasterio(moa_lowres)

moa_highres = DATA_DIR + '/surface_imagery/MODIS_MOA/2014/moa125_2014_hp1_v01.tif' 
moa_highres_da = rioxarray.open_rasterio(moa_highres)

Warning 1: TIFFReadDirectory:Invalid data type for tag StripByteCounts
Warning 1: TIFFFetchNormalTag:Incorrect value for "GeoKeyDirectory"; tag ignored
Warning 1: TIFFFetchNormalTag:ASCII value for tag "GeoASCIIParams" contains null byte in value; value incorrectly truncated during reading due to implementation limitations


# Pre-process data

In [48]:
# Clip altimetry datasets to grounding line to limit analysis to only below grounded ice
CS2_Smith2017.rio.write_crs(3031, inplace=True)
CS2_Smith2017 = CS2_Smith2017.rio.clip(Scripps_landice.geometry.values, Scripps_landice.crs, drop=False)
ATL15_dh.rio.write_crs("epsg:3031", inplace=True)
ATL15_dh = ATL15_dh.rio.clip(Scripps_landice.geometry.values, Scripps_landice.crs)

In [49]:
dataset1=CS2_Smith2017
dataset2=ATL15_dh

# Create empty lists to store data
cyc_start_dates = []
cyc_end_dates = []
midcyc_dates = []

for idx in range(len(dataset1.delta_h[:33])):
    # Smith and others, 2017 method CryoSat-2 SARIn data
    if dataset1.identifier_product_DOI == 'doi:10.5194/tc-11-451-2017':
        cyc_start_date = dataset1.time.values[idx]
        cyc_end_date = dataset1.time.values[idx+1]
        midcyc_days = cyc_end_date - cyc_start_date
        midcyc_date = cyc_start_date + midcyc_days/2
        cyc_start_dates += [cyc_start_date]
        cyc_end_dates += [cyc_end_date]
        midcyc_dates += [midcyc_date]
    # Cryo-TEMPO-EOLIS Swath Thematic Gridded Product 
    elif dataset1.Title == 'Land Ice Elevation Thematic Gridded Product':
        date_time_str = '70-01-01'
        date_time_obj = datetime.datetime.strptime(date_time_str, '%y-%m-%d')
        cyc_start_date = date_time_obj + datetime.timedelta(seconds = ds_sub.time.values[idx].astype(float))
        cyc_end_date = date_time_obj + datetime.timedelta(seconds = ds_sub.time.values[idx+1].astype(float))
        midcyc_days = cyc_end_date - cyc_start_date
        midcyc_date = cyc_start_date + midcyc_days/2
        cyc_start_dates += [cyc_start_date]
        cyc_end_dates += [cyc_end_date]
        midcyc_dates += [midcyc_date]
for idx in range(len(dataset2.delta_h)-1):
    # ICESat-2 ATL15 r003
    if dataset2.identifier_product_DOI == '10.5067/ATLAS/ATL15.003':    
        cyc_start_date = dataset2.time.values[idx]
        cyc_end_date = dataset2.time.values[idx+1]
        midcyc_days = cyc_end_date - cyc_start_date
        midcyc_date = cyc_start_date + midcyc_days/2
        cyc_start_dates += [cyc_start_date]
        cyc_end_dates += [cyc_end_date]
        midcyc_dates += [midcyc_date]

In [160]:
# Ensure dates are correct
for idx in range(len(midcyc_dates)):
    if idx <= 32:
        print('idx:', idx, 'CS2 era')
        print('cyc_start_date:', CS2_Smith2017.time[idx].values)
        print('mid_cyc_date:', midcyc_dates[idx])
        print('cyc_end_date:', CS2_Smith2017.time[idx+1].values)
    elif idx > 32:
        print('idx:', idx, 'IS2 era')
        # Subtract 33 (32th idx because 0th idx) from idx to start over with new dataset
        print('cyc_start_date:', ATL15_dh.time[idx-33].values)
        print('mid_start_date:', midcyc_dates[idx])
        print('cyc_end_date:', ATL15_dh.time[(idx+1)-33].values)

idx: 0 CS2 era
cyc_start_date: 2010-07-02T15:00:00.000000000
mid_cyc_date: 2010-08-17T06:45:00.000000000
cyc_end_date: 2010-10-01T22:30:00.000000000
idx: 1 CS2 era
cyc_start_date: 2010-10-01T22:30:00.000000000
mid_cyc_date: 2010-11-16T11:15:00.000000000
cyc_end_date: 2011-01-01T00:00:00.000000000
idx: 2 CS2 era
cyc_start_date: 2011-01-01T00:00:00.000000000
mid_cyc_date: 2011-02-15T15:45:00.000000000
cyc_end_date: 2011-04-02T07:30:00.000000000
idx: 3 CS2 era
cyc_start_date: 2011-04-02T07:30:00.000000000
mid_cyc_date: 2011-05-17T23:15:00.000000000
cyc_end_date: 2011-07-02T15:00:00.000000000
idx: 4 CS2 era
cyc_start_date: 2011-07-02T15:00:00.000000000
mid_cyc_date: 2011-08-17T06:45:00.000000000
cyc_end_date: 2011-10-01T22:30:00.000000000
idx: 5 CS2 era
cyc_start_date: 2011-10-01T22:30:00.000000000
mid_cyc_date: 2011-11-16T11:15:00.000000000
cyc_end_date: 2012-01-01T00:00:00.000000000
idx: 6 CS2 era
cyc_start_date: 2012-01-01T00:00:00.000000000
mid_cyc_date: 2012-02-15T15:45:00.000000000
c

In [ ]:
# IS IT WORTH TO GO THRU EFFORT TO COMBINE S09 and SF18 lakes

In [ ]:
# PERHAPS MOVE THE CELLS BELOW TO SAUTHOFF2024.ipynb/.py?

In [76]:
# View head of Smith and others, 2009 static outline geodataframe
Smith2009_outlines.head()

,Name,Description,geometry,geometry_2d
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...","POLYGON ((-792264.327 -691480.857, -793834.289..."
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...","POLYGON ((-842788.063 -708464.240, -842354.948..."
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...","POLYGON ((-874893.221 -654533.044, -872997.487..."
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...","POLYGON ((-828821.778 -584874.415, -828370.520..."
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...","POLYGON ((-858067.460 -573467.564, -858714.391..."


In [77]:
Smith2009_outlines[Smith2009_outlines.Name == 'Recovery_8']

,Name,Description,geometry,geometry_2d
98,Recovery_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-74016.771 896653.746 1.000, -7350...","POLYGON ((-74016.771 896653.746, -73507.172 89..."


In [78]:
SiegfriedFricker2018_SF18outlines

,name,geometry,area (m^2),perimeter (m),cite
0,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,93967.290533,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
1,Cook_E2,"POLYGON ((765592.392 -1714713.856, 765682.174 ...",2.680933e+08,127109.861123,"McMillan and others, 2013, GRL, doi:10.1002/gr..."
2,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,122225.431584,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
3,KT1,"POLYGON ((-556189.687 -681400.000, -556000.000...",4.549884e+07,31575.761008,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
4,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,32351.217554,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
5,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,33296.288468,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
6,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,21525.193986,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
7,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,31339.673859,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
8,Lake78,"MULTIPOLYGON (((-257325.009 -525800.047, -2571...",2.330027e+08,128470.660267,"Carter and others, 2013, J. Glac., doi:10.3189..."
9,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,64534.427498,"Fricker and others, 2010, J. Glac., doi:10.318..."


In [79]:
# Find outlines in SiegfriedFricker2018_outlines static outline geodataframe with Smith and others, 2009 citation
# These lakes have not been redefined with a new outline and can be analysed as is
SiegfriedFricker2018_S09outlines = SiegfriedFricker2018_outlines[SiegfriedFricker2018_outlines['cite'].isin(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'])]
SiegfriedFricker2018_S09outlines

,name,geometry,area (m^2),perimeter (m),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,51147.562479,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,37249.152584,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,44183.483257,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,62680.016773,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,73686.203194,"Smith and others, 2009, J. Glac., doi:10.3189/..."
...,...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,31952.842516,"Smith and others, 2009, J. Glac., doi:10.3189/..."
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,32373.996995,"Smith and others, 2009, J. Glac., doi:10.3189/..."
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,45873.974279,"Smith and others, 2009, J. Glac., doi:10.3189/..."
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,89565.314574,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [80]:
# Find outlines in SiegfriedFricker2018_outlines geodataframe with citations not matching Smith and others, 2009
# These are eitherly newly identified lakes or lakes that have been redefined with new static outlines 
SiegfriedFricker2018_SF18outlines = SiegfriedFricker2018_outlines[~SiegfriedFricker2018_outlines['cite'].isin(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'])]
SiegfriedFricker2018_SF18outlines

,name,geometry,area (m^2),perimeter (m),cite
23,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,93967.290533,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
25,Cook_E2,"POLYGON ((765592.392 -1714713.856, 765682.174 ...",2.680933e+08,127109.861123,"McMillan and others, 2013, GRL, doi:10.1002/gr..."
41,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,122225.431584,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
65,KT1,"POLYGON ((-556189.687 -681400.000, -556000.000...",4.549884e+07,31575.761008,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,32351.217554,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,33296.288468,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
80,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,21525.193986,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
81,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,31339.673859,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
82,Lake78,"MULTIPOLYGON (((-257325.009 -525800.047, -2571...",2.330027e+08,128470.660267,"Carter and others, 2013, J. Glac., doi:10.3189..."
85,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,64534.427498,"Fricker and others, 2010, J. Glac., doi:10.318..."


In [81]:
# Find spatial intersections between two static outline geodataframes to indicate groups of outlines that are redefined outlines of one lake
S09_SF18_intersections = find_intersections(Smith2009_outlines, SiegfriedFricker2018_outlines)

In [82]:
SiegfriedFricker2018_outlines.iloc[98]

name                                                      Ninnis_1
geometry         POLYGON ((1120237.97298 -1921686.74163, 112223...
area (m^2)                                        188719447.956114
perimeter (m)                                         50658.748856
cite             Smith and others, 2009, J. Glac., doi:10.3189/...
Name: 98, dtype: object

In [83]:
S09_SF18_intersections

[(0, 0),
 (0, 91),
 (1, 1),
 (1, 92),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (13, 14),
 (14, 8),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (19, 19),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 26),
 (24, 27),
 (25, 28),
 (26, 29),
 (27, 30),
 (28, 31),
 (29, 32),
 (30, 33),
 (31, 34),
 (32, 35),
 (33, 36),
 (34, 37),
 (35, 38),
 (36, 39),
 (37, 40),
 (38, 43),
 (39, 44),
 (40, 45),
 (41, 46),
 (42, 47),
 (43, 48),
 (44, 49),
 (45, 42),
 (46, 50),
 (47, 51),
 (48, 52),
 (49, 53),
 (50, 54),
 (51, 55),
 (52, 56),
 (53, 57),
 (54, 58),
 (55, 59),
 (56, 60),
 (57, 61),
 (58, 62),
 (59, 63),
 (60, 64),
 (61, 65),
 (62, 69),
 (63, 70),
 (64, 71),
 (65, 68),
 (66, 72),
 (67, 73),
 (68, 74),
 (69, 75),
 (70, 76),
 (71, 77),
 (72, 78),
 (73, 79),
 (74, 83),
 (75, 84),
 (76, 85),
 (77, 86),
 (78, 88),
 (78, 89),
 (79, 90),
 (80, 82),
 (81, 93),
 (82, 94),
 (83, 95),
 (84, 96),
 (85, 97),
 (86, 98),
 (87, 99),
 (88, 100),
 (89, 109

In [84]:
# Extract indicies from each geodataframe for later use
Smith2009_intersection_indices = [inner_list[0] for inner_list in S09_SF18_intersections]
SiegfriedFricker2018_intersection_indices = [inner_list[1] for inner_list in S09_SF18_intersections]

In [92]:
# Print the lake names from Smith and others, 2009 and Siegfried & Fricker, 2018 that intersect
# Print 'name' column for rows in gdf1 corresponding to indices in indices1
series1 = Smith2009_outlines.iloc[Smith2009_intersection_indices]['Name']

# Print 'name' column for rows in gdf2 corresponding to indices in indices2
series2 = SiegfriedFricker2018_outlines.iloc[SiegfriedFricker2018_intersection_indices]['name']

lake_names = pd.concat([series1.reset_index(drop=True), series2.reset_index(drop=True)], axis=1)
lake_names.columns = ['S09', 'SF18']

# Filtering rows where column1 and column2 differ
filtered_df = lake_names[lake_names['S09'] != lake_names['SF18']]

print(filtered_df)

                S09                       SF18
1    Bindschadler_1                       Mac7
3    Bindschadler_2                       Mac8
59     InstituteE_1               Institute_E1
60     InstituteE_2               Institute_E2
61     InstituteW_1               Institute_W1
62     InstituteW_2               Institute_W2
63      KambTrunk_1                        KT1
78       Macayeal_1                       Mac1
79       Macayeal_2                       Mac2
80       Macayeal_3                       Mac4
81       Macayeal_3                       Mac5
82       Macayeal_4                       Mac6
83         Mercer_1                     Lake78
84         Mercer_2       MercerSubglacialLake
92      Recovery_10                       Rec9
93      Recovery_11                      Rec10
94       Recovery_1                       Rec1
95       Recovery_2                       Rec1
96       Recovery_3                       Rec2
97       Recovery_4                       Rec3
98       Reco

In [62]:
print(Smith2009_outlines.Name.to_string())

0      Bindschadler_1
1      Bindschadler_2
2      Bindschadler_3
3      Bindschadler_4
4      Bindschadler_5
5      Bindschadler_6
6              Byrd_1
7              Byrd_2
8            Byrd_s10
9            Byrd_s11
10           Byrd_s12
11           Byrd_s13
12           Byrd_s14
13           Byrd_s15
14            Byrd_s1
15            Byrd_s2
16            Byrd_s3
17            Byrd_s4
18            Byrd_s5
19            Byrd_s6
20            Byrd_s7
21            Byrd_s8
22            Byrd_s9
23            David_1
24           David_s1
25           David_s2
26           David_s3
27           David_s4
28           David_s5
29              EAP_1
30              EAP_2
31              EAP_3
32              EAP_4
33              EAP_5
34              EAP_6
35              EAP_7
36              EAP_8
37              EAP_9
38      Foundation_10
39      Foundation_11
40      Foundation_12
41      Foundation_13
42      Foundation_14
43      Foundation_15
44      Foundation_16
45       F

In [172]:
# Extract rows with intersection between the Smith and others, 2009 static outlines and the static outlines in Siegfried & Fricker, 2018
# These are lakes that were defined in Smith and others, 2009 and subsequently renamed or redelineated by publications after Smith and others, 2009
# We will analyze as a unified polygon of all past static outlines for each lake
Smith2009_intersections = Smith2009_outlines[Smith2009_outlines.index.isin(Smith2009_intersection_indices)] 
# Smith2009_intersections

In [176]:
# Do the same for the Siegfried & Fricker, 2018 static outlines
SiegfriedFricker2018_intersections = SiegfriedFricker2018_outlines[SiegfriedFricker2018_outlines.index.isin(SiegfriedFricker2018_intersection_indices)] 
SiegfriedFricker2018_intersections

,name,geometry,area (m^2),perimeter (m),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,51147.562479,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,37249.152584,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,44183.483257,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,62680.016773,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,73686.203194,"Smith and others, 2009, J. Glac., doi:10.3189/..."
...,...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,31952.842516,"Smith and others, 2009, J. Glac., doi:10.3189/..."
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,32373.996995,"Smith and others, 2009, J. Glac., doi:10.3189/..."
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,45873.974279,"Smith and others, 2009, J. Glac., doi:10.3189/..."
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,89565.314574,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [161]:
# Extract rows with no intersection between the Smith and others, 2009 and the renamed or redefined static outlines in Siegfried & Fricker, 2018
# These are new lakes that were delineated by Smith and others, 2009 and not subsequently rename or redelineated
# These lakes only have one static outline and can be analysed as is
Smith2009_non_intersections = Smith2009_outlines[~Smith2009_outlines.index.isin(Smith2009_intersection_indices)] 
# Smith2009_non_intersections

In [165]:
len(Smith2009_non_intersections)

96

In [158]:
len(SiegfriedFricker2018_non_intersections) # new lakes

7

In [159]:
len(SiegfriedFricker2018_intersections) # redefined lakes

27

In [164]:
SiegfriedFricker2018_outlines.head()

,name,geometry,area (m^2),perimeter (m),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,51147.562479,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,37249.152584,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,44183.483257,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,62680.016773,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,73686.203194,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [166]:
len(SiegfriedFricker2018_outlines)

131

In [163]:
SiegfriedFricker2018_intersections

,name,geometry,area (m^2),perimeter (m),cite
23,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,93967.290533,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
25,Cook_E2,"POLYGON ((765592.392 -1714713.856, 765682.174 ...",2.680933e+08,127109.861123,"McMillan and others, 2013, GRL, doi:10.1002/gr..."
41,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,122225.431584,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
65,KT1,"POLYGON ((-556189.687 -681400.000, -556000.000...",4.549884e+07,31575.761008,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
80,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,21525.193986,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
81,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,31339.673859,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
82,Lake78,"MULTIPOLYGON (((-257325.009 -525800.047, -2571...",2.330027e+08,128470.660267,"Carter and others, 2013, J. Glac., doi:10.3189..."
85,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,64534.427498,"Fricker and others, 2010, J. Glac., doi:10.318..."
86,Mac2,"POLYGON ((-657250.020 -890550.040, -657249.980...",1.457161e+08,57055.433906,"Fricker and others, 2010, J. Glac., doi:10.318..."
88,Mac4,"POLYGON ((-739324.970 -857799.960, -739199.980...",7.520909e+07,38826.874743,"Fricker and others, 2010, J. Glac., doi:10.318..."


In [177]:
# Extract rows with no intersection between the Smith and others, 2009 and the renamed or redefined static outlines in Siegfried & Fricker, 2018
# These are new lakes that were added by publications subsequent to Smith and others, 2009
# These lakes only have one static outline and can be analysed as is
SiegfriedFricker2018_non_intersections = SiegfriedFricker2018_outlines[~SiegfriedFricker2018_outlines.index.isin(SiegfriedFricker2018_intersection_indices)] 
SiegfriedFricker2018_non_intersections

,name,geometry,area (m^2),perimeter (m),cite
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,32351.217554,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,33296.288468,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
87,Mac3,"POLYGON ((-642325.010 -883925.000, -642449.980...",1.502500e+08,104150.365454,"Fricker and others, 2010, J. Glac., doi:10.318..."
117,Thw_124,"POLYGON ((-1419402.000 -431819.000, -1421479.0...",5.761346e+08,112135.767376,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."
118,Thw_142,"POLYGON ((-1404705.000 -410106.000, -1404692.0...",1.607689e+08,57828.708474,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."
119,Thw_170,"POLYGON ((-1380997.000 -398714.000, -1382313.0...",1.940960e+08,54559.367618,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."
120,Thw_70,"POLYGON ((-1476557.000 -449098.000, -1474606.0...",3.540275e+08,89548.375571,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."


In [71]:
# View lakes manually retrieved from other papers
Sauthoff2024_S24outlines

,name,geometry,area (m^2),perimeter (m),cite,CS2_SARIn
0,L1,POINT (1273770.727 -1199082.221),NaN,NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False
1,LowerMercerSubglacialLake,POINT (-308667.100 -509669.900),NaN,NaN,"Siegfried and Fricker, 2021, Geophys. Res. Let...",True
2,LowerSubglacialLakeConway,POINT (-283073.400 -504430.900),NaN,NaN,"Siegfried and Fricker, 2021, Geophys. Res. Let...",True
3,Recovery_8,"POLYGON ((-74016.771 896653.746, -73507.172 89...",NaN,NaN,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
4,TL108,POINT (-1427500.000 -542000.000),NaN,NaN,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
5,TL115,POINT (-1422500.000 -537500.000),NaN,NaN,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
6,TL122,POINT (-1419000.000 -532500.000),NaN,NaN,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
7,TL96,POINT (-1439000.000 -544000.000),NaN,NaN,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
8,U1,POINT (1085448.426 -1085448.426),NaN,NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False
9,U2,POINT (1040017.716 -1068353.903),NaN,NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False


# Analysis

In [78]:
# Isolate first listed lake from Siegfried & Fricker, 2018 inventory
lake_gdf=SiegfriedFricker2018_outlines.iloc[1:2]
lake_gdf

,name,geometry,area (m^2),perimeter (m),cite
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,37249.152584,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [52]:
# Plot height change time series
height_change_time_series(lake_gdf, CS2_Smith2017, ATL15_dh)

Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/height_change_time_series' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/height_change_time_series/Bindschadler_1' already exists.
Now creating/saving plots...
Complete


In [63]:
# Find evolving outliens
gdf = find_evolving_outlines(
    lake_gdf=lake_gdf, area_multiple=3, level=0.1, dataset1=CS2_Smith2017, dataset2=ATL15_dh)
gdf

Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines/Bindschadler_1' already exists.
Now creating/saving plots...
Complete


,geometry,area (m^2),dh (m),vol (m^3),midcyc_datetime,centroid
0,"POLYGON ((-789525.995 -690126.091, -790528.980...",5.553003e+06,0.109543,6.082917e+05,2023-02-15 21:45:00,POINT (-790614.866 -691481.865)
1,"POLYGON ((-787520.025 -687691.580, -788523.010...",2.922731e+06,0.103719,3.031430e+05,2023-02-15 21:45:00,POINT (-787746.065 -688859.459)
2,"POLYGON ((-796952.397 -704501.589, -796546.889...",1.089847e+07,-0.138468,-1.509093e+06,2023-02-15 21:45:00,POINT (-793408.856 -707189.981)
3,"POLYGON ((-784002.349 -706493.899, -784024.964...",3.995020e+06,-0.111918,-4.471151e+05,2023-02-15 21:45:00,POINT (-785309.731 -707706.921)
4,"POLYGON ((-779183.926 -702509.279, -779496.145...",2.427501e+06,-0.107391,-2.606926e+05,2023-02-15 21:45:00,POINT (-780556.497 -703834.044)
...,...,...,...,...,...,...
87,"POLYGON ((-781033.783 -707490.054, -781238.505...",1.424128e+06,-0.108612,-1.546774e+05,2023-02-15 21:45:00,POINT (-780535.299 -706291.001)
88,"POLYGON ((-783807.153 -691551.575, -783797.966...",7.734631e+06,-0.127457,-9.858302e+05,2023-02-15 21:45:00,POINT (-785534.741 -692093.580)
89,"POLYGON ((-794540.920 -697272.752, -795543.904...",2.849566e+06,0.105499,3.006271e+05,2023-02-15 21:45:00,POINT (-794424.994 -698169.653)
90,"POLYGON ((-779496.145 -690037.244, -780096.923...",1.040736e+07,0.110596,1.151009e+06,2023-02-15 21:45:00,POINT (-780354.535 -692910.990)


In [64]:
# Define lake name and polygon
lake_poly = lake_gdf.iloc[0].geometry
area_multiple = 2
buffered_gdf = muliple_area_buffer(lake_poly, area_multiple)

In [66]:
# Check which evolving outline polygons touch or cross the evolving outline search limit polygon
# Check if geometries in left_gdf touches or crosses the boundary of the right_geom
within = gpd.sjoin(gdf, buffered_gdf, predicate='within')
overlaps = gpd.sjoin(gdf, buffered_gdf, predicate='overlaps')

# Find percentage intersecting
intersecting_percent = np.round((len(within) / (len(within) + len(overlaps))),2)
intersecting_percent

0.57

In [ ]:
# Repeat process until percentage is greater than 90%


# Analysis of previously identified lakes

In [49]:
for idx in range(129, len(SiegfriedFricker2018_outlines)):
    print(idx)
    # Isolate lake from Siegfried & Fricker, 2018 inventory as geodataframe using slicing
    lake_gdf=SiegfriedFricker2018_outlines.iloc[idx:idx+1]
    
    # Plot data counts
    data_counts(lake_gdf=lake_gdf, dataset1=CS2_Smith2017, dataset2=ATL15_dh)

    # Plot plan view height change with various area multiple polygons
    # Made it through 40
    height_change_time_series(lake_gdf=lake_gdf, dataset1=CS2_Smith2017, dataset2=ATL15_dh)
    
    # Clear the output of each index
    clear_output(wait=True)

130
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/data_counts' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/data_counts/Wilkes_2' created successfully.
Now creating/saving plots...
Complete
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/height_change_time_series' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/height_change_time_series/Wilkes_2' created successfully.
Now creating/saving plots...
Complete


In [ ]:
# Do same for lake points in SF24


In [ ]:
# After this step, plan view height change plots with area multiple polygons were visually inspected
# Plots were inspected to determine appropriate area multiple to use for finding evolving outlines

In [ ]:
# By default lake analysis used an area multiple of three to generate evolving outlines
# and an area multiple of two was used as the limit of evolving outline search when 
# finding the percentage of evolving outlines that intersected with the limit of investigation
# A larger area multiple was used to generate the evolving outlines such that some percentage
# would intersect with the limit of investigation to determine the adjustments to the level used to
# generate the outlines

In [50]:
# Find evolving outlines
idx=0


lake_gdf=SiegfriedFricker2018_outlines.iloc[idx:idx+1]
gdf = find_evolving_outlines(
    lake_gdf=lake_gdf, area_multiple=3, level=0.1, dataset1=CS2_Smith2017, dataset2=ATL15_dh)

Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines/Bindschadler_1' already exists.
Now creating/saving plots...
Complete


,geometry,area (m^2),dh (m),vol (m^3),midcyc_datetime,centroid
0,"POLYGON ((-789536.618 -690093.135, -790544.909...",5.609027e+06,0.109543,6.144288e+05,2023-02-15 21:45:00,POINT (-790631.250 -691455.381)
1,"POLYGON ((-787520.036 -687647.003, -788528.327...",2.952220e+06,0.103719,3.062016e+05,2023-02-15 21:45:00,POINT (-787747.272 -688820.457)
2,"POLYGON ((-797002.307 -704537.254, -796594.654...",1.100841e+07,-0.138468,-1.524317e+06,2023-02-15 21:45:00,POINT (-793440.020 -707238.478)
3,"POLYGON ((-783983.751 -706539.074, -784006.486...",4.035322e+06,-0.111542,-4.501068e+05,2023-02-15 21:45:00,POINT (-785298.049 -707757.886)
4,"POLYGON ((-779139.838 -702535.434, -779453.709...",2.451992e+06,-0.107391,-2.633227e+05,2023-02-15 21:45:00,POINT (-780519.670 -703866.522)
...,...,...,...,...,...,...
88,"POLYGON ((-780999.481 -707539.984, -781205.287...",1.438495e+06,-0.108612,-1.562379e+05,2023-02-15 21:45:00,POINT (-780498.360 -706335.208)
89,"POLYGON ((-783787.522 -691525.423, -783778.287...",7.812669e+06,-0.127457,-9.957767e+05,2023-02-15 21:45:00,POINT (-785524.249 -692070.016)
90,"POLYGON ((-794578.073 -697273.910, -795586.363...",2.878312e+06,0.105499,3.036598e+05,2023-02-15 21:45:00,POINT (-794461.534 -698175.093)
91,"POLYGON ((-779453.709 -690003.865, -780057.665...",1.051237e+07,0.110596,1.162623e+06,2023-02-15 21:45:00,POINT (-780316.640 -692891.328)


In [51]:
# Define lake name and polygon
lake_poly = lake_gdf.iloc[0].geometry
area_multiple = 2
buffered_gdf = muliple_area_buffer(lake_poly, area_multiple)

In [52]:
# Check which evolving outline polygons touch or cross the evolving outline search limit polygon
# Check if geometries in left_gdf touches or crosses the boundary of the right_geom
within = gpd.sjoin(gdf, buffered_gdf, predicate='within')
overlaps = gpd.sjoin(gdf, buffered_gdf, predicate='overlaps')

# Find percentage intersecting
intersecting_percent = np.round((len(within) / (len(within) + len(overlaps))),2)
intersecting_percent

0.58

In [ ]:
idx=8
level = 0.1  # Starting level
within_percent = 0  # Initial intersecting_percent
area_multiple_search_extent = 2
area_multiple_search_extent_plusone = area_multiple_search_extent + 1

while within_percent < 0.90:
    lake_gdf = SiegfriedFricker2018_outlines.iloc[idx:idx+1]
    gdf = find_evolving_outlines(lake_gdf=lake_gdf, 
        area_multiple=area_multiple_search_extent_plusone, level=level, 
        # dataset1=CS2_Smith2017, dataset2=ATL15_dh)
        dataset1='none', dataset2=ATL15_dh)
    
    # Define lake polygon
    lake_poly = lake_gdf.iloc[0].geometry
    
    # Define buffered geodataframe as the limit of search extent
    buffered_gdf = muliple_area_buffer(lake_poly, area_multiple_search_extent)
    
    # Check which evolving outlines are within or overlap the evolving outline search limit
    within = gpd.sjoin(gdf, buffered_gdf, predicate='within')
    overlaps = gpd.sjoin(gdf, buffered_gdf, predicate='overlaps')
    
    # Find percentage within
    if (len(within) + len(overlaps)) > 0:  # Prevent division by zero
        within_percent = np.round((len(within) / (len(within) + len(overlaps))), 2)
    else:
        break    
    
    print(f"Level: {level}, Within percent: {within_percent}")
    
    # Check if the desired condition is met
    if within_percent >= 0.90:
        break  # Exit the loop if condition is met
    
    level += 0.01  # Increase level for next iteration
    level = np.round(level, 2)  # Ensure level is rounded to two decimal places
    
# Play sound to indicate complete   
play_sound()

Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines/Byrd_s1' already exists.
Now creating/saving plots...
Complete
Level: 0.1, Within percent: 0.26
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines/Byrd_s1' already exists.
Now creating/saving plots...
Complete
Level: 0.11, Within percent: 0.32
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines' already exists.
Folder '/home/jovyan/1_outlines_candidates/output/FigS1_lake_reexamination_methods.ipynb/find_evolving_outlines/Byrd_s1' already exists.
Now creating/saving plots...
Complete
Level: 0.12, Within p

In [ ]:
# Plot outlines
plot_evolving_outlines(gdf)

In [ ]:
# Clean up outlines
filtered_gdf = extract_intersecting_polygons(gdf, lake_gdf['geometry'].values[0]).copy()

In [ ]:
# Plot outlines
plot_evolving_outlines(filtered_gdf)

In [ ]:
# Simplify centroid column to string to allow GeoJSON export
for column in filtered_gdf.columns:
    if column != 'geometry':  # Exclude the geometry column
        filtered_gdf[column] = filtered_gdf[column].astype(str)

# Export evolving outlines GeoDataFrame to GeoJSON
filtered_gdf.to_file(filename='evolving_outlines/{}.geojson'.format(lake_gdf['name'].values[0]), driver='GeoJSON')
# gdf.to_file(filename='evolving_outlines/test.geojson', driver='GeoJSON')

In [121]:
# Test opening works
gdf = gpd.read_file('evolving_outlines/{}.geojson'.format(lake_gdf['name'].values[0]))
gdf

DriverError: evolving_outlines/Byrd_s1.geojson: No such file or directory

In [ ]:
# The following lakes needed a non-default area multiples for this analysis:
# 4x area multiple for evolving outline search extent
lakes_4 = ['Byrd_2', ]

In [ ]:
# The following lakes had partial CS2 SARIn coverage, so CS2 was occluded from analysis
lakes_4 = ['Byrd_s1',]

In [ ]:
SiegfriedFricker2018_S09outlines

In [ ]:
SiegfriedFricker2018_non_intersections

In [ ]:
# create list of S09 lakes redelineated in SF18
S09_SF18_redefined_lake_names = [   
    ['Cook_E2', 'Cook_E2'],
    ['KambTrunk_1', 'KT1'],
    ['Macayeal_1', 'Mac1'],
    ['Macayeal_2', 'Mac2'],
    ['Mercer_1', 'Lake78'],
    ['Mercer_2', 'MercerSubglacialLake'],
    ['Recovery_3', 'Rec2'],
    ['Recovery_4', 'Rec3'],
    ['Recovery_5', 'Rec4'], 
    ['Recovery_6', 'Rec5'],
    ['Recovery_7', 'Rec6'],
    ['Recovery_9', 'Rec8'],
    ['Recovery_10', 'Rec9'],
    ['Recovery_11', 'Rec10'],     
    ['Whillans_1', 'EngelhardtSubglacialLake'],            
    ['Whillans_2a', 'Lake12'],
    ['Whillans_2b', 'Lake10'],
    ['Whillans_3', 'WhillansSubglacialLake'],
    ['Whillans_4', 'ConwaySubglacialLake'],
    ['Whillans_5', 'UpperSubglacialLakeConway'],
    # Two S09 lakes converted to one SF18 lake
    ['Slessor_2', 'Slessor_3', 'Slessor_23'],
    ['Recovery_1', 'Recovery_2', 'Rec1'],
    # One S09 lake converted to two SF18 lakes
    ['Macayeal_3', 'Mac4', 'Mac5']
]

In [ ]:
Smith2009_outlines[Smith2009_outlines['Name'] == 'Cook_E2']['geometry_2d'],
SiegfriedFricker2018_outlines[SiegfriedFricker2018_outlines['name'] == 'Cook_E2']['geometry']],

# Fig. S1

In [ ]:
ROI_poly = buffered_poly
print(type(buffered_poly))
print(type(ROI_poly))

In [ ]:
ROI_poly = buffered_poly
dataset1 = CS2_dh
dataset2 = ATL15_dh

# Clipping datasets
dataset1_clipped = dataset1.rio.clip(ROI_poly, dataset1.rio.crs)
dataset2_clipped = dataset2.rio.clip(ROI_poly, dataset2.rio.crs)

# Extract min and max of x and y for dataset1
min_x1 = dataset1_clipped.x.min().item()
max_x1 = dataset1_clipped.x.max().item()
min_y1 = dataset1_clipped.y.min().item()
max_y1 = dataset1_clipped.y.max().item()

# Extract min and max of x and y for dataset2
min_x2 = dataset2_clipped.x.min().item()
max_x2 = dataset2_clipped.x.max().item()
min_y2 = dataset2_clipped.y.min().item()
max_y2 = dataset2_clipped.y.max().item()

# Check if the coordinates match
if min_x1 != min_x2 or max_x1 != max_x2 or min_y1 != min_y2 or max_y1 != max_y2:
    raise ValueError("Dataset1 and Dataset2 do not have matching x, y min, max coordinates")

# Establish x_min, x_max, y_min, y_max, 
x_min = min_x1
x_max = max_x1
y_min = min_y1
y_max = max_y1

# Subsetting datasets
# Subset datasets to region of interest for plotting
buffer = 4000
mask_x = (dataset1.x >= x_min-buffer) & (dataset1.x <= x_max+buffer)
mask_y = (dataset1.y >= y_min-buffer) & (dataset1.y <= y_max+buffer)
dataset1_subset = dataset1.where(mask_x & mask_y, drop=True)
mask_x = (dataset2.x >= x_min-buffer) & (dataset2.x <= x_max+buffer)
mask_y = (dataset2.y >= y_min-buffer) & (dataset2.y <= y_max+buffer)
dataset2_subset = dataset2.where(mask_x & mask_y, drop=True)
mask_x = (moa_highres_da.x >= x_min-buffer) & (moa_highres_da.x <= x_max+buffer)
mask_y = (moa_highres_da.y >= y_min-buffer) & (moa_highres_da.y <= y_max+buffer)
moa_highres_da_subset = moa_highres_da.where(mask_x & mask_y, drop=True)

In [ ]:
# Modify find contours function to achieve something like less than 5% of contours intersecting with buffer
# gdf = find_evolving_outlines(ROI['name'].values[0], buffered_poly, 0.5, CS2_dh, ATL15_dh)
# gdf = find_evolving_outlines(ROI['name'].values[0], buffered_poly, 0.5, CS2_dh, ATL15_dh)
gdf = find_evolving_outlines('Slessor_2_3_23', buffered_poly, 0.5, CS2_dh, ATL15_dh)

In [ ]:
# Remove off-lake outlines (find function in lake areas notebook)
gdf_subset = remove_outlier_polygon(gdf, 'y', 'max')

In [ ]:
gdf_subset = remove_outlier_polygon(gdf_subset, 'y', 'max')

In [ ]:
gdf_subset = remove_outlier_polygon(gdf_subset, 'y', 'max')

In [ ]:
gdf_subset = remove_outlier_polygon(gdf_subset, 'y', 'min')

In [ ]:
gdf_subset = remove_outlier_polygon(gdf_subset, 'y', 'min')

In [ ]:
# TODO
# Try other lakes

# Plot Fig. 2
fig, ax = plt.subplots(3,1, sharex=True, figsize=(5.5,16.5))


# Panel A - Plot uplift filling event
# Specify the time value you want to plot
specified_date = datetime.date(2010, 8, 17)
gdf_subset[(gdf_subset['date'] == specified_date) & (gdf_subset['dh (m)'] > 0)].boundary.plot(ax=ax[0], color='blue')
gdf_subset[(gdf_subset['date'] == specified_date) & (gdf_subset['dh (m)'] < 0)].boundary.plot(ax=ax[0], color='red')

# Calculate the absolute difference between each time in the dataset and the specified time
time_diff = np.abs(midcyc_dates - np.datetime64(specified_date))
# Find the index of the minimum difference
nearest_time_index = time_diff.argmin().item()
if nearest_time_index <= 32:
    dhdt = dataset1_subset.cyc_to_cyc_delta_h[nearest_time_index,:,:]
elif nearest_time_index > 32:
    # Subtract 33 from idx to start over with new dataset
    dhdt = dataset2_subset.cyc_to_cyc_delta_h[(nearest_time_index-33),:,:]

# Plot gridded height change data
divnorm=colors.TwoSlopeNorm(vmin=-1.5, vcenter=0., vmax=1.5)  
img = ax[0].imshow(dhdt, extent=[x_min-buffer, x_max+buffer, y_min-buffer, y_max+buffer], origin='upper', cmap='coolwarm_r', 
                   # norm=colors.CenteredNorm(),
                   norm=divnorm)

# Plot buffered polygon showing extent of evolving outline search
ROI_color = 'magenta'
gpd.GeoSeries(buffered_poly).boundary.plot(ax=ax[0], color=ROI_color)

# Create an axes on the right side of ax1 for the colorbar
cax = fig.add_axes([ax[0].get_position().x1 + 0.15, ax[0].get_position().y0 - 0.092, 0.03, ax[0].get_position().height])
fig.colorbar(img, cax=cax).set_label('height change [m]', size=12)

# Annotate time slice
ax[0].annotate('height change: {} to {}'.format(datetime64_to_fractional_year(cyc_start_dates[nearest_time_index]),
    datetime64_to_fractional_year(cyc_end_dates[nearest_time_index])), 
    xy=(-421e3,1009e3), xycoords='data', fontsize=14)


# Panel B - Plot subsidence draining event
# specified_date = datetime.date(2021, 2, 15)
specified_date = datetime.date(2020, 2, 16)
# specified_date = datetime.date(2015, 8, 17)

gdf_subset[(gdf_subset['date'] == specified_date) & (gdf_subset['dh (m)'] > 0)].boundary.plot(ax=ax[1], color='blue')
gdf_subset[(gdf_subset['date'] == specified_date) & (gdf_subset['dh (m)'] < 0)].boundary.plot(ax=ax[1], color='red')

# Calculate the absolute difference between each time in the dataset and the specified time
time_diff = np.abs(midcyc_dates - np.datetime64(specified_date))
# Find the index of the minimum difference
nearest_time_index = time_diff.argmin().item()
if nearest_time_index <= 32:
    dhdt = dataset1_subset.delta_h[nearest_time_index+1,:,:]-dataset1_subset.delta_h[nearest_time_index,:,:]
elif nearest_time_index > 32:
    # Subtract 33 from idx to start over with new dataset
    dhdt = dataset2_subset.delta_h[(nearest_time_index-33)+1,:,:]-dataset2_subset.delta_h[(nearest_time_index-33),:,:]

# Plot gridded height change data
img = ax[1].imshow(dhdt, extent=[x_min-buffer, x_max+buffer, y_min-buffer, y_max+buffer], origin='lower', cmap='coolwarm_r', 
                   # norm=colors.CenteredNorm(),
                   norm=divnorm)
gpd.GeoSeries(buffered_poly).boundary.plot(ax=ax[1], color=ROI_color)

ax[1].annotate('height change: {} to {}'.format(datetime64_to_fractional_year(cyc_start_dates[nearest_time_index]),
    datetime64_to_fractional_year(cyc_end_dates[nearest_time_index])), 
    xy=(-421e3,1009e3), xycoords='data', fontsize=14)


# Panel C - Plot outlines in aggregate vs. two past static delineations
# Plot MOA imagery  
ax[2].imshow(moa_highres_da_subset[0,:,:], cmap="gray", clim=[14000, 17000], extent=[x_min-buffer, x_max+buffer, y_min-buffer, y_max+buffer])

# Pick colormap and make continuous cmap discrete for evolving outlines
colormap = 'plasma'
continuous_cmap = matplotlib.colormaps[colormap]
discrete_cmap = colors.ListedColormap(continuous_cmap(np.linspace(0, 1, len(midcyc_dates)-1)))

# Norm to time variable
norm = plt.Normalize(mdates.date2num(midcyc_dates[0]), 
                     mdates.date2num(midcyc_dates[-1]))

# Use for loop to store each time slice as line segment to use in legend
# And plot each outline in the geopandas dataframe and color by date
lines = []  # list of lines to be used for the legend
for idx, dt in enumerate(midcyc_dates, 0):
    x = 1; y = 1
    line, = ax[2].plot(x, y, color=discrete_cmap(norm(mdates.date2num(midcyc_dates[idx]))), linewidth=3)
    lines.append(line)
    
    # Filter rows that match the current time slice
    gdf_subset_dt = gdf_subset[gdf_subset['datetime'] == dt]

    # Plotting the subset
    gdf_subset_dt.plot(ax=ax[2], edgecolor=discrete_cmap(norm(mdates.date2num(midcyc_dates[idx]))), facecolor='none')

    
# All panels
# Label axes and set limits
ax[2].set_xlabel('x [km]', size=16)
ax[1].set_ylabel('y [km]', size=16)

# ax[0].annotate('A', xy=(-425e3,1049e3), xycoords='data', fontsize=30)
# ax[1].annotate('B', xy=(-425e3,1049e3), xycoords='data', fontsize=30)
# ax[2].annotate('C', xy=(-425e3,1049e3), xycoords='data', fontsize=30)
ax[0].annotate('A', xy=(0.02, 0.9), xycoords='axes fraction', fontsize=30)
ax[1].annotate('B', xy=(0.02, 0.9), xycoords='axes fraction', fontsize=30)
ax[2].annotate('C', xy=(0.02, 0.9), xycoords='axes fraction', fontsize=30)

# Create lines for legend
S09_color = 'lightseagreen'
SF18_color = 'teal'
Smith2009 = plt.Line2D((0, 1), (0, 0), color=S09_color, linestyle=(0, (1, 2)), linewidth=3)
SiegfriedFricker2018 = plt.Line2D((0, 1), (0, 0), color=SF18_color, linestyle=(0, (1, 1)), linewidth=3)
ROI = plt.Line2D((0, 1), (0, 0), color=ROI_color, linestyle='solid', linewidth=3)
uplift = plt.Line2D((0, 1), (0, 0), color='blue', linewidth=3)
subsidence = plt.Line2D((0, 1), (0, 0), color='red', linewidth=3)

# Create legends
ax[0].legend([Smith2009, SiegfriedFricker2018, ROI, uplift], 
           ['static outline [10]',
            'static outline [13]', 
            # 'evolving outline ({} m threshold)'.format(threshold)], 
            'evolving outline search limit [this study]',
            'evolving outline - uplift [this study]'], 
             loc='upper right') 

ax[1].legend([subsidence],
           ['evolving outline - subsidence [this study]'], 
             loc='upper right')

legend = ax[2].legend([tuple(lines)], ['evolving outlines [this study]'],
    handlelength=3, handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
    loc='upper center')
legend.get_frame().set_linewidth(0.0)
ax[2].patch.set_alpha(1)

# Create colorbar 
m = plt.cm.ScalarMappable(cmap=discrete_cmap)
m.set_array(np.array([datetime64_to_fractional_year(date) for date in midcyc_dates[0:]]))
cax = inset_axes(ax[2],
                 width="100%",
                 height="2.5%",
                 loc=3,
                 bbox_to_anchor=[0,-0.14,1,1],
                 bbox_transform=ax[2].transAxes,
                 borderpad=0,
                 )
cbar=fig.colorbar(m, ticks=np.array([2010,2012,2014,2016,2018,2020,2022]), 
             cax=cax, orientation='horizontal').set_label('evolving outline year', size=15)

# Plot inset map
axIns = ax[0].inset_axes([0.01, 0.01, 0.3, 0.3]) # [left, bottom, width, height] (fractional axes coordinates)
axIns.set_aspect('equal')
moa_2014_coastline.plot(ax=axIns, color='gray', edgecolor='k', linewidth=0.1, zorder=3)
moa_2014_groundingline.plot(ax=axIns, color='ghostwhite', edgecolor='k', linewidth=0.1, zorder=3)
axIns.axis('off')
# # Plot black rectangle to indicate location
# rect = Rectangle((x_min, y_min), (x_max-x_min), (y_max-y_min), fill=False, linewidth=2, color='k', zorder=3)
# axIns.add_artist(rect)
# Plot red star to indicate location
axIns.scatter(((x_max+x_min)/2), ((y_max+y_min)/2), marker='*', 
    linewidth=1, edgecolor='k', facecolor='r', s=100, zorder=3)

# # Add annotation to the opposite side of the colorbar
# cbar.ax.text(1.1, 0.5, 'CryoSat-2 era', va='center', ha='left', transform=cbar.ax.transAxes)

for i in ax: 
    Smith2009_outlines.boundary.plot(ax=i, edgecolor=S09_color, facecolor='none', linestyle=(0, (1, 2)), linewidth=3, alpha=1, zorder=0)
    SiegfriedFricker2018_SF18outlines.boundary.plot(ax=i, edgecolor=SF18_color, facecolor='none', linestyle=(0, (1, 1)), linewidth=3, alpha=1, zorder=0)
    
    # Change polar stereographic m to km
    km_scale = 1e3
    ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
    i.xaxis.set_major_formatter(ticks_x)
    ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/km_scale))
    i.yaxis.set_major_formatter(ticks_y)
    
    # i.set(xlim=(x_min-buffer, x_max+buffer), ylim=(y_min-buffer, y_max+buffer))   
    i.set(xlim=(x_min-buffer, x_max+buffer), ylim=(y_min-buffer, y_max+buffer))   

plt.tight_layout()

plt.show()